In [1]:
#from geopy.distance import geodesic
from metar import Metar
from utils.data_treatment import parse_wind_info, get_metar_features, get_voos_para_o_destino_no_mesmo_horario, fill_na, estatistical_features
from datetime import datetime, timedelta
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
# Specify the file path or URL to load the CSV file
file_path = './../data/public.csv'

# Load the CSV file into a Pandas DataFrame
df = pd.read_csv(file_path)
df

,flightid,hora_ref,origem,destino,url_img_satelite,metaf,metar,prev_troca_cabeceira,troca_cabeceira_hora_anterior,espera
0,504a62621cd231d6ab67e674ce538cd3,2022-06-01T01:00:00Z,SBCF,SBFL,http://satelite.cptec.inpe.br/repositoriogoes/...,NaN,METAR SBFL 010000Z 17009KT 140V200 9999 BKN030...,0,1,0.0
1,b0fd0f83644625ecc21f5261e8e5e347,2022-06-01T01:00:00Z,SBPA,SBFL,http://satelite.cptec.inpe.br/repositoriogoes/...,NaN,METAR SBFL 010000Z 17009KT 140V200 9999 BKN030...,0,1,0.0
2,1210f0ca07ddca00d09a3e02d3b100d8,2022-06-01T01:00:00Z,SBSP,SBCF,http://satelite.cptec.inpe.br/repositoriogoes/...,NaN,METAR SBCF 010000Z 12006KT CAVOK 21/14 Q1018=,0,0,0.0
3,b25032f34507cce285ee779446496568,2022-06-01T01:00:00Z,SBGR,SBCT,http://satelite.cptec.inpe.br/repositoriogoes/...,NaN,METAR SBCT 010000Z 10006KT 7000 -RA BKN004 OVC...,0,0,0.0
4,00762a9892ecba7c66d1d87800d38cac,2022-06-01T01:00:00Z,SBSP,SBSV,http://satelite.cptec.inpe.br/repositoriogoes/...,NaN,METAR SBSV 010000Z 11008KT 9999 FEW023 27/21 Q...,0,1,0.0
...,...,...,...,...,...,...,...,...,...,...
302394,8ca8c4d16e592d65cd25d341113aecdf,2023-05-31T23:00:00Z,SBCT,SBSP,http://satelite.cptec.inpe.br/repositoriogoes/...,METAF SBSP 312300Z 16004KT 2000 BR OVC033...,METAR SBSP 312200Z 20007KT 170V230 9999 SCT009...,0,0,NaN
302395,afde50d413c2374ab53cd3101332f9fc,2023-05-31T23:00:00Z,SBCT,SBKP,http://satelite.cptec.inpe.br/repositoriogoes/...,METAF SBKP 312300Z 11006KT 5000 BR SCT024...,METAR SBKP 312200Z 20004KT 9999 OVC035 18/16 Q...,0,0,NaN
302396,faeb2f6e744c606aee9e819ea1d62e6b,2023-05-31T23:00:00Z,SBGL,SBKP,http://satelite.cptec.inpe.br/repositoriogoes/...,METAF SBKP 312300Z 11006KT 5000 BR SCT024...,METAR SBKP 312200Z 20004KT 9999 OVC035 18/16 Q...,0,0,NaN
302397,3992d063be713b35d3bf46b79853880e,2023-05-31T23:00:00Z,SBKP,SBRJ,http://satelite.cptec.inpe.br/repositoriogoes/...,METAF SBRJ 312300Z 08012KT 9999 FEW033 ...,METAR SBRJ 312200Z 31004KT 240V340 9999 FEW007...,0,0,NaN


In [3]:
df['origem - destino'] = [f"{row.origem}_{row.destino}" for _, row in df.iterrows()]

for pos, metar in enumerate(df.metar):
    if isinstance(metar, float):
        df.loc[pos, "metar"] = fill_na(pos, df)

In [4]:
extracted_data = []
for idx, row in tqdm(df.iterrows()):   
    if isinstance(row.metar, str): 
        try:
            data = Metar.Metar(row.metar)
        except:
            print("Error getting metar row ", idx)
            data = np.nan
        
        if not isinstance(data, float):
            extracted_data.append(get_metar_features(data))
        else:
            extracted_data.append(data)
    else:
        extracted_data.append(row.metar)

nan_keys = {}
for key in extracted_data[0].keys():
    nan_keys[key] = None

for idx, row in enumerate(extracted_data):   
    if isinstance(extracted_data[idx], float): 
        extracted_data[idx] = nan_keys

df = pd.concat([df, pd.DataFrame(extracted_data)], axis=1)
df

298873it [00:45, 7017.41it/s]

Error getting metar row  297594
Error getting metar row  297633


302399it [00:45, 6604.63it/s]


,flightid,hora_ref,origem,destino,url_img_satelite,metaf,metar,prev_troca_cabeceira,troca_cabeceira_hora_anterior,espera,...,min_temp_6hr,max_temp_24hr,min_temp_24hr,precip_1hr,precip_3hr,precip_6hr,precip_24hr,ice_accretion_1hr,ice_accretion_3hr,ice_accretion_6hr
0,504a62621cd231d6ab67e674ce538cd3,2022-06-01T01:00:00Z,SBCF,SBFL,http://satelite.cptec.inpe.br/repositoriogoes/...,NaN,METAR SBFL 010000Z 17009KT 140V200 9999 BKN030...,0,1,0.0,...,None,None,None,None,None,None,None,None,None,None
1,b0fd0f83644625ecc21f5261e8e5e347,2022-06-01T01:00:00Z,SBPA,SBFL,http://satelite.cptec.inpe.br/repositoriogoes/...,NaN,METAR SBFL 010000Z 17009KT 140V200 9999 BKN030...,0,1,0.0,...,None,None,None,None,None,None,None,None,None,None
2,1210f0ca07ddca00d09a3e02d3b100d8,2022-06-01T01:00:00Z,SBSP,SBCF,http://satelite.cptec.inpe.br/repositoriogoes/...,NaN,METAR SBCF 010000Z 12006KT CAVOK 21/14 Q1018=,0,0,0.0,...,None,None,None,None,None,None,None,None,None,None
3,b25032f34507cce285ee779446496568,2022-06-01T01:00:00Z,SBGR,SBCT,http://satelite.cptec.inpe.br/repositoriogoes/...,NaN,METAR SBCT 010000Z 10006KT 7000 -RA BKN004 OVC...,0,0,0.0,...,None,None,None,None,None,None,None,None,None,None
4,00762a9892ecba7c66d1d87800d38cac,2022-06-01T01:00:00Z,SBSP,SBSV,http://satelite.cptec.inpe.br/repositoriogoes/...,NaN,METAR SBSV 010000Z 11008KT 9999 FEW023 27/21 Q...,0,1,0.0,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
302394,8ca8c4d16e592d65cd25d341113aecdf,2023-05-31T23:00:00Z,SBCT,SBSP,http://satelite.cptec.inpe.br/repositoriogoes/...,METAF SBSP 312300Z 16004KT 2000 BR OVC033...,METAR SBSP 312200Z 20007KT 170V230 9999 SCT009...,0,0,NaN,...,None,None,None,None,None,None,None,None,None,None
302395,afde50d413c2374ab53cd3101332f9fc,2023-05-31T23:00:00Z,SBCT,SBKP,http://satelite.cptec.inpe.br/repositoriogoes/...,METAF SBKP 312300Z 11006KT 5000 BR SCT024...,METAR SBKP 312200Z 20004KT 9999 OVC035 18/16 Q...,0,0,NaN,...,None,None,None,None,None,None,None,None,None,None
302396,faeb2f6e744c606aee9e819ea1d62e6b,2023-05-31T23:00:00Z,SBGL,SBKP,http://satelite.cptec.inpe.br/repositoriogoes/...,METAF SBKP 312300Z 11006KT 5000 BR SCT024...,METAR SBKP 312200Z 20004KT 9999 OVC035 18/16 Q...,0,0,NaN,...,None,None,None,None,None,None,None,None,None,None
302397,3992d063be713b35d3bf46b79853880e,2023-05-31T23:00:00Z,SBKP,SBRJ,http://satelite.cptec.inpe.br/repositoriogoes/...,METAF SBRJ 312300Z 08012KT 9999 FEW033 ...,METAR SBRJ 312200Z 31004KT 240V340 9999 FEW007...,0,0,NaN,...,None,None,None,None,None,None,None,None,None,None


In [17]:
df.temperature = np.float32(df.temperature)
df.dew_point = np.float32(df.dew_point)
df.wind_velocity = np.float32(df.wind_velocity)
df.visibility = np.float32(df.visibility)
df.pressure = np.float32(df.pressure)
df.few_cloud_height = np.float32(df.few_cloud_height)
df.broken_cloud_height = np.float32(df.broken_cloud_height)
df.overcast_cloud_height = np.float32(df.overcast_cloud_height)
df.scattered_cloud_height = np.float32(df.scattered_cloud_height)

In [85]:
features = estatistical_features(df, 'origem - destino', pd.Series.count, None, '1h', None)

features.rename(columns={'entity': 'origem - destino'}, inplace=True)
df = pd.merge(df, features, on=['origem - destino', 'hora_ref'], how='inner')
df

array([None, '1300', '2300', '4000', '300', '2000', '5000', '1000',
       '1500', '2100', '700', '500', '900', '800', '2500', '3000', '1200',
       '3500', '200', '2600', '2700', '100', '0', '1600', '1400', '1700',
       '1100', '1900', '4500', '400', '2200', '3700', '600', '4900',
       '1800', '3300', '3600', '2400', '4800', '4600', '4200', '4300',
       '2800', '3200', '3100', '4700', '6000', '2900', '3800', '5500',
       '3900', '5100', '5300', '4400', '3400'], dtype=object)

In [25]:
def count_distinct(df):
    return len(df.unique())

#entitys = ['origem - destino', "origem", "destino", "destino", "destino", "destino", "destino", "destino", "destino", "destino", "destino", "destino", "destino"]
#over_columns = [None, None, None, "temperature", "dew_point", "wind_velocity", "visibility",  "pressure", "present_weather", "few_cloud_height", "broken_cloud_height", "overcast_cloud_height", "scattered_cloud_height"]
#time_windows = [["1h", "3h", "6h", "12h", "7d", "30d", "90d"], ["1h", "3h", "6h", "12h", "7d", "30d", "90d"], ["12h", "7d", "30d", "90d"], ["1h", "3h", "6h", "12h"], ["1h", "3h", "6h", "12h"], ["1h", "3h", "6h", "12h"], ["1h", "3h", "6h", "12h"], ["1h", "3h", "6h", "12h"], ["1h", "3h", "6h", "12h"], ["1h", "3h", "6h", "12h"], ["1h", "3h", "6h", "12h"], ["1h", "3h", "6h", "12h"], ["1h", "3h", "6h", "12h"]]
#transformations = [["count"], ["count"], ["count"], ["avg", "max", "min"], ["avg", "max", "min"], ["avg", "max", "min"], ["avg", "max", "min"], ["avg", "max", "min"], ["count_distinct"], ["avg", "max", "min"], ["avg", "max", "min"], ["avg", "max", "min"], ["avg", "max", "min"]]

entitys = ["destino"]
over_columns = ["prev_troca_cabeceira", "troca_cabeceira_hora_anterior"]
time_windows = [["1h", "3h", "6h", "12h"], ["1h", "3h", "6h", "12h"]]
transformations = [["count", "avg"], ["count", "avg"]]

for pos, e in enumerate(entitys):
    for time in time_windows[pos]:
        for fnct in transformations[pos]:
            print(e, over_columns[pos], time, fnct)

            if fnct == "count":
                fun = pd.Series.count
            elif fnct == "avg":
                fun = pd.Series.mean
            elif fnct == "max":
                fun = pd.Series.max
            elif fnct == "min":
                fun = pd.Series.min
            elif fnct == "count_distinct":
                fun = count_distinct

            features = estatistical_features(df, e, fun, over_columns[pos], time, None)
            features.rename(columns={'entity': e}, inplace=True)
            df = pd.merge(df, features, on=[e, 'hora_ref'], how='inner')
df.to_csv('data.csv', index=False) 
df

destino prev_troca_cabeceira 1h count
Number of Entities:  12
Iteration 0 of 12
Iteration 1 of 12
Iteration 2 of 12
Iteration 3 of 12
Iteration 4 of 12
Iteration 5 of 12
Iteration 6 of 12
Iteration 7 of 12
Iteration 8 of 12
Iteration 9 of 12
Iteration 10 of 12
Iteration 11 of 12
destino prev_troca_cabeceira 1h avg
Number of Entities:  12
Iteration 0 of 12
Iteration 1 of 12
Iteration 2 of 12
Iteration 3 of 12
Iteration 4 of 12
Iteration 5 of 12
Iteration 6 of 12
Iteration 7 of 12
Iteration 8 of 12
Iteration 9 of 12
Iteration 10 of 12
Iteration 11 of 12
destino prev_troca_cabeceira 3h count
Number of Entities:  12
Iteration 0 of 12
Iteration 1 of 12
Iteration 2 of 12
Iteration 3 of 12
Iteration 4 of 12
Iteration 5 of 12
Iteration 6 of 12
Iteration 7 of 12
Iteration 8 of 12
Iteration 9 of 12
Iteration 10 of 12
Iteration 11 of 12
destino prev_troca_cabeceira 3h avg
Number of Entities:  12
Iteration 0 of 12
Iteration 1 of 12
Iteration 2 of 12
Iteration 3 of 12
Iteration 4 of 12
Iteration 5 

,flightid,hora_ref,origem,destino,url_img_satelite,metaf,metar,prev_troca_cabeceira,troca_cabeceira_hora_anterior,espera,...,max_destino_over_scattered_cloud_height_past_12h,min_destino_over_scattered_cloud_height_past_12h,count_destino_over_prev_troca_cabeceira_past_1h,mean_destino_over_prev_troca_cabeceira_past_1h,count_destino_over_prev_troca_cabeceira_past_3h,mean_destino_over_prev_troca_cabeceira_past_3h,count_destino_over_prev_troca_cabeceira_past_6h,mean_destino_over_prev_troca_cabeceira_past_6h,count_destino_over_prev_troca_cabeceira_past_12h,mean_destino_over_prev_troca_cabeceira_past_12h
0,504a62621cd231d6ab67e674ce538cd3,2022-06-01 01:00:00+00:00,SBCF,SBFL,http://satelite.cptec.inpe.br/repositoriogoes/...,NaN,METAR SBFL 010000Z 17009KT 140V200 9999 BKN030...,0,1,0.0,...,NaN,NaN,3,0.0,3,0.0,3,0.0,3,0.000000
1,00762a9892ecba7c66d1d87800d38cac,2022-06-01 01:00:00+00:00,SBSP,SBSV,http://satelite.cptec.inpe.br/repositoriogoes/...,NaN,METAR SBSV 010000Z 11008KT 9999 FEW023 27/21 Q...,0,1,0.0,...,NaN,NaN,5,0.0,5,0.0,5,0.0,5,0.000000
2,2c6941c4b747277aab4dd189fb23e1a4,2022-06-01 01:00:00+00:00,SBRF,SBGR,http://satelite.cptec.inpe.br/repositoriogoes/...,METAF SBGR 010100Z 11008KT 1000 BR OVC033...,METAR SBGR 010000Z 13005KT 9999 OVC009 16/15 Q...,0,0,0.0,...,NaN,NaN,3,0.0,3,0.0,3,0.0,3,0.000000
3,2bc132d5cfb2de55bdfbb6437c3ef7f0,2022-06-01 01:00:00+00:00,SBGR,SBBR,http://satelite.cptec.inpe.br/repositoriogoes/...,NaN,METAR SBBR 010000Z 07002KT CAVOK 21/08 Q1018=,0,0,0.0,...,NaN,NaN,4,0.0,4,0.0,4,0.0,4,0.000000
4,65d4824bcd8253bf5b14e62d02d386f4,2022-06-01 01:00:00+00:00,SBSP,SBCT,http://satelite.cptec.inpe.br/repositoriogoes/...,NaN,METAR SBCT 010000Z 10006KT 7000 -RA BKN004 OVC...,0,0,0.0,...,NaN,NaN,4,0.0,4,0.0,4,0.0,4,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
302394,8ca8c4d16e592d65cd25d341113aecdf,2023-05-31 23:00:00+00:00,SBCT,SBSP,http://satelite.cptec.inpe.br/repositoriogoes/...,METAF SBSP 312300Z 16004KT 2000 BR OVC033...,METAR SBSP 312200Z 20007KT 170V230 9999 SCT009...,0,0,NaN,...,1100.0,900.0,12,0.0,36,0.0,66,0.0,144,0.187500
302395,afde50d413c2374ab53cd3101332f9fc,2023-05-31 23:00:00+00:00,SBCT,SBKP,http://satelite.cptec.inpe.br/repositoriogoes/...,METAF SBKP 312300Z 11006KT 5000 BR SCT024...,METAR SBKP 312200Z 20004KT 9999 OVC035 18/16 Q...,0,0,NaN,...,2300.0,300.0,4,0.0,11,0.0,20,0.0,38,0.157895
302396,faeb2f6e744c606aee9e819ea1d62e6b,2023-05-31 23:00:00+00:00,SBGL,SBKP,http://satelite.cptec.inpe.br/repositoriogoes/...,METAF SBKP 312300Z 11006KT 5000 BR SCT024...,METAR SBKP 312200Z 20004KT 9999 OVC035 18/16 Q...,0,0,NaN,...,2300.0,300.0,4,0.0,11,0.0,20,0.0,38,0.157895
302397,3992d063be713b35d3bf46b79853880e,2023-05-31 23:00:00+00:00,SBKP,SBRJ,http://satelite.cptec.inpe.br/repositoriogoes/...,METAF SBRJ 312300Z 08012KT 9999 FEW033 ...,METAR SBRJ 312200Z 31004KT 240V340 9999 FEW007...,0,0,NaN,...,3000.0,500.0,14,0.0,32,0.0,58,0.0,116,0.189655
